Stuff to do:
Make sure all images in 4 page papers are recieved (Done... I think)
Test on other collections
Clean up program
Implement OCR

In [615]:
import requests
import time
import ast
import random
import os
import sys
import re
from bs4 import BeautifulSoup

import numpy as np
import PIL
from PIL import Image

In [616]:
def remove_duplicates(str_list: list) -> list:
    
    ret_list = []
    for item in str_list:
        if item not in ret_list:
            ret_list.append(item)
    return ret_list

In [617]:
#Function takes in list of hrefs to pages and gets page IDs
def get_page_ids(href_list: list) -> list:
    
    ret_list = []
    for page in href_list:
        temp_list = []
        n = random.randint(6, 10)
        time.sleep(n)
        result = requests.get(page)
        
        print(result)
                
        if result.status_code == 200:
            soup = BeautifulSoup(result.content, "html.parser")
            
        print("==================== SOUP ===================")
        print(soup)
        print("\n\n\n\nData:")

        data = soup.find_all('script')
        print(data)
        script_text = str(data[5])
        print('\n\n================== SCRIPT TEXT =================\n\n\n')
        print(script_text)
        
        pid_list = [m.start() for m in re.finditer('"pid":"0,', script_text)]

        for pid in pid_list:
            i = script_text.find('",', pid)
            pid_num = script_text[pid+9:i]
            temp_list.append(pid_num)
        
        ret_list.append(remove_duplicates(temp_list))
        
    return ret_list

In [618]:
# Gets href list
def get_links(url: str) -> list:
    page = url
    
    n = random.randint(6, 10)
    time.sleep(n)
    
    result = requests.get(page)
    print('[attemping to reach source code at:' + url + ']')
    print(str(result))
    if result.status_code == 200:
        print("[source code found.]")
        soup = BeautifulSoup(result.content, "html.parser")
    else:
        print("[source code not found.]")
        return [[],[]]

    data = soup.find_all('script')

    script_text = str(data[2])

    i = script_text.find('summary_data')
    j = script_text.rfind(']')

    my_list = script_text[i+14:j+1]

    cool_list = ast.literal_eval(my_list) 

    ret_list = []
    href_list = []
    img_list = []
    
    for year in cool_list:
        if 'editions' in year:
            year_list = year['editions']
            for edition in year_list:
                img_list.append(edition['img_url'])
                href_list.append(edition['href_url'])
                
    ret_list.append(href_list)
    ret_list.append(img_list)
    
    return ret_list

In [619]:
def get_news_id(url: str) -> str:
    i = url.find("?id=")
    j = url.find("&s")
    return url[i+4:j]
    

In [620]:
def get_template_img_urls(url: str) -> list:
    list_of_hrefs = get_links(url)
    pid_list = get_page_ids(list_of_hrefs[0])

    final_array = []

    for i in range(len(list_of_hrefs[1])):
        print("\n\n\nCurrently getting template url for image at link:")
        link = list_of_hrefs[1][i]
        print(list_of_hrefs[0][i])
        print(link)
        print("The following are the PIDS for this page:")
        print(pid_list[i])
        print("Here are the created urls:")
        print("=====================")
        for j in range(len(pid_list[i])):
            url1 = "https://news.google.com/newspapers?id=" + get_news_id(link) + "&sjid=HioMAAAAIBAJ&pg=0%2C" + pid_list[i][j] + "&img=1&hl=en&zoom=3&tid="
            final_array.append(url1)
            print(url1)
        print("=====================")

    return final_array

In [621]:
def get_file(image_url: str) -> str:
    print("[Retrieving image from URL]")
    n = random.randint(6, 10)
    time.sleep(n)
    img_data = requests.get(image_url).content
    
    data_str = str(img_data)
    if data_str.startswith("b'GIF"):
        print("Image is a gif.")
        return ""
    
    else:
        print("Image is not a gif!")
        with open("output/" + image_url[-5:] + "cool_image.jpg", "wb") as handler:
            handler.write(img_data)
        return "output/" + image_url[-5:] + "cool_image.jpg"

def generate_images(url: str) -> list:
    all_imgs = []
    found_end = False
    i = 0
    while found_end == False and i < 50:
        send_url = url + str(i)
        ret = get_file(send_url)
        if ret == "":
            found_end = True
        else:
            found_end = False
            all_imgs.append(ret)
        i+=1
    return all_imgs

In [622]:
def get_concat_h_blank(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v_blank(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def get_concat_h_multi_blank(im_list: list, dr: str):
    _im = im_list.pop(0)
    for im in im_list:
        if dr == "h":
            _im = get_concat_h_blank(_im, im)
        else:
            _im = get_concat_v_blank(_im, im)
    return _im

def save_h_imgs_35(obj_list: list) -> None:
    get_concat_h_multi_blank([obj_list[0], obj_list[1], obj_list[2], obj_list[9], obj_list[10]], 'h').save('h1.jpg')
    get_concat_h_multi_blank([obj_list[3], obj_list[4], obj_list[5], obj_list[11], obj_list[12]], 'h').save('h2.jpg')
    get_concat_h_multi_blank([obj_list[6], obj_list[7], obj_list[8], obj_list[13], obj_list[14]], 'h').save('h3.jpg')
    get_concat_h_multi_blank([obj_list[15], obj_list[16], obj_list[17], obj_list[24], obj_list[25]], 'h').save('h4.jpg')
    get_concat_h_multi_blank([obj_list[18], obj_list[19], obj_list[20], obj_list[26], obj_list[27]], 'h').save('h5.jpg')
    get_concat_h_multi_blank([obj_list[21], obj_list[22], obj_list[23], obj_list[28], obj_list[29]], 'h').save('h6.jpg')  
    get_concat_h_multi_blank([obj_list[30], obj_list[31], obj_list[32], obj_list[33], obj_list[34]], 'h').save('h7.jpg') 
    
def save_h_imgs_48(obj_list: list) -> None:
    get_concat_h_multi_blank([obj_list[0], obj_list[1], obj_list[2], obj_list[9], obj_list[10], obj_list[11]], 'h').save('h1.jpg')
    get_concat_h_multi_blank([obj_list[3], obj_list[4], obj_list[5], obj_list[12], obj_list[13], obj_list[14]], 'h').save('h2.jpg')
    get_concat_h_multi_blank([obj_list[6], obj_list[7], obj_list[8], obj_list[15], obj_list[16], obj_list[17]], 'h').save('h3.jpg')
    get_concat_h_multi_blank([obj_list[18], obj_list[19], obj_list[20], obj_list[27], obj_list[28], obj_list[29]], 'h').save('h4.jpg')
    get_concat_h_multi_blank([obj_list[21], obj_list[22], obj_list[23], obj_list[30], obj_list[31], obj_list[32]], 'h').save('h5.jpg')  
    get_concat_h_multi_blank([obj_list[24], obj_list[25], obj_list[26], obj_list[33], obj_list[34], obj_list[35]], 'h').save('h6.jpg') 
    get_concat_h_multi_blank([obj_list[36], obj_list[37], obj_list[38], obj_list[42], obj_list[43], obj_list[44]], 'h').save('h7.jpg')
    get_concat_h_multi_blank([obj_list[39], obj_list[40], obj_list[41], obj_list[45], obj_list[46], obj_list[47]], 'h').save('h8.jpg')
    
def save_v_img(n: int, num: int):
    """ Given integer n, saves files named in the format
    'h' + n + '.jpg' as a large image stacked together. Named
    with num. """
    
    saved_obj_list = []
    for i in range(1, n):
        im = Image.open("h" + str(i) + ".jpg")
        saved_obj_list.append(im)

    get_concat_h_multi_blank(saved_obj_list, 'v').save('v' + str(num) + '.jpg')


def create_newspaper(n: int, img_url: str):
    """ Given img_url, generates a newspaper
    from images scraped from that url. """
    
    my_list = generate_images(img_url)
    #my_list = ['output/tid=0cool_image.jpg', 'output/tid=1cool_image.jpg', 'output/tid=2cool_image.jpg', 'output/tid=3cool_image.jpg', 'output/tid=4cool_image.jpg', 'output/tid=5cool_image.jpg', 'output/tid=6cool_image.jpg', 'output/tid=7cool_image.jpg', 'output/tid=8cool_image.jpg', 'output/tid=9cool_image.jpg', 'output/id=10cool_image.jpg', 'output/id=11cool_image.jpg', 'output/id=12cool_image.jpg', 'output/id=13cool_image.jpg', 'output/id=14cool_image.jpg', 'output/id=15cool_image.jpg', 'output/id=16cool_image.jpg', 'output/id=17cool_image.jpg', 'output/id=18cool_image.jpg', 'output/id=19cool_image.jpg', 'output/id=20cool_image.jpg', 'output/id=21cool_image.jpg', 'output/id=22cool_image.jpg', 'output/id=23cool_image.jpg', 'output/id=24cool_image.jpg', 'output/id=25cool_image.jpg', 'output/id=26cool_image.jpg', 'output/id=27cool_image.jpg', 'output/id=28cool_image.jpg', 'output/id=29cool_image.jpg', 'output/id=30cool_image.jpg', 'output/id=31cool_image.jpg', 'output/id=32cool_image.jpg', 'output/id=33cool_image.jpg', 'output/id=34cool_image.jpg', 'output/id=35cool_image.jpg', 'output/id=36cool_image.jpg', 'output/id=37cool_image.jpg', 'output/id=38cool_image.jpg', 'output/id=39cool_image.jpg', 'output/id=40cool_image.jpg', 'output/id=41cool_image.jpg', 'output/id=42cool_image.jpg', 'output/id=43cool_image.jpg', 'output/id=44cool_image.jpg', 'output/id=45cool_image.jpg', 'output/id=46cool_image.jpg', 'output/id=47cool_image.jpg']
    obj_list = []
    for file in my_list:
        im = Image.open(file)
        obj_list.append(im)
    
    list_len = len(my_list)
    if list_len == 35:
        save_h_imgs_35(obj_list)
        save_v_img(8, n)
    elif list_len == 48:
        save_h_imgs_48(obj_list)
        save_v_img(9, n)
        
    for item in my_list:
        os.remove(item)
    

In [623]:
def get_newspapers(url: str) -> None:
    """ Generates newspapers as jpgs located
    at the url on google's newspaper archives."""
    
    list_of_imgs = get_template_img_urls(url)
    i = 1
    for img in list_of_imgs:
        print("Looking at img: " + img)
        print('[generating images]')
        #create_newspaper(i, img)
        i+=1


In [624]:
#get_newspapers("https://news.google.com/newspapers?nid=F_tUKv7nyWgC")
print(get_page_ids(["https://news.google.com/newspapers?id=CzYbAAAAIBAJ&sjid=hUoEAAAAIBAJ&pg=3056,119259&img=1&zoom=1"]))

[attemping to reach source code at:https://news.google.com/newspapers?nid=F_tUKv7nyWgC]
<Response [200]>
[source code found.]
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>



Currently getting template url for image at link:
https://news.google.com/newspapers?nid=F_tUKv7nyWgC&dat=17750112&printsec=frontpage&hl=en
https://news.google.com/newspapers?id=CzYbAAAAIBAJ&

In [599]:
#1227806
#1232784
#1238419
#1243520